In [ ]:
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def get_neighbors(L):
    """Generate neighbor indices for a 2D lattice with periodic boundary conditions."""
    lattice = np.arange(L*L).reshape(L, L)
    ups = np.roll(lattice, -1, axis=0)
    rights = np.roll(lattice, -1, axis=1)
    downs = np.roll(lattice, 1, axis=0)
    lefts = np.roll(lattice, 1, axis=1)
    return ups, rights, downs, lefts

def montecarlo(L, T, nsweeps=10000000, measure_rate=5000):
    """
    Run Monte Carlo simulation for the 2D Ising model WITHOUT thermalization.
    This version collects configurations starting from sweep 0 (non-equilibrium).
    
    Parameters:
    - L: Lattice size (L x L)
    - T: Temperature
    - nsweeps: Total number of sweeps
    - measure_rate: Frequency of configuration measurements
    
    Returns:
    - List of spin configurations (includes non-equilibrium samples!)
    """
    beta = 1/T
    conf = np.random.choice([-1, 1], (L, L))  # Random initial state
    confs = []  # NO thermalization - collects from beginning!
    ups, rights, downs, lefts = get_neighbors(L)

    for sweep in range(nsweeps):
        for idx in np.ndindex(conf.shape):
            i, j = idx
            deltaE = 2.0 * conf[i,j] * (conf[ups[i,j] // L, ups[i,j] % L] +
                                        conf[rights[i,j] // L, rights[i,j] % L] +
                                        conf[downs[i,j] // L, downs[i,j] % L] +
                                        conf[lefts[i,j] // L, lefts[i,j] % L])
            # Metropolis criterion
            if deltaE <= 0 or np.random.random() < np.exp(-beta * deltaE):
                conf[i, j] *= -1  # Flip spin

        # Store configurations from the start (NO thermalization period)
        if sweep % measure_rate == 0:
            confs.append(conf.copy())

    return confs

def generate_ising_dataset(L=20, n_configs_per_temp=100):
    """
    Generate Ising model configurations WITHOUT thermalization.
    WARNING: This will produce biased, non-equilibrium data!
    
    Parameters:
    - L: Lattice size
    - n_configs_per_temp: Number of configurations to generate per temperature
    
    Returns:
    - X: Array of flattened configurations (n_samples, L*L)
    - y: Array of labels (0=ordered, 1=disordered)
    - temperatures: Array of temperatures used
    """
    Tc = 2.269  # Critical temperature for 2D Ising model
    
    # Define temperature ranges
    # Below Tc: ordered phase
    T_ordered = np.linspace(1.5, 2.1, 6)
    # Near Tc: critical region
    T_critical = np.linspace(2.15, 2.4, 3)
    # Above Tc: disordered phase
    T_disordered = np.linspace(2.5, 3.5, 6)
    
    all_temps = np.concatenate([T_ordered, T_critical, T_disordered])
    
    X = []
    y = []
    temperatures = []
    
    start_time = datetime.now()
    print("="*60)
    print("WARNING: NO THERMALIZATION - COLLECTING NON-EQUILIBRIUM DATA")
    print("="*60)
    print(f"Started data generation: {start_time.strftime('%Y-%m-%d %H:%M')}")
    print(f"Critical temperature Tc = {Tc}")
    print(f"Generating {n_configs_per_temp} configurations at each of {len(all_temps)} temperatures")
    print(f"Lattice size: {L} x {L}\n")
    
    for idx, T in enumerate(all_temps):
        print(f"Temperature {idx+1}/{len(all_temps)}: T = {T:.3f}", end=" ")
        
        # Determine label
        if T < Tc:
            label = 0  # Ordered
            phase = "ordered"
        else:
            label = 1  # Disordered
            phase = "disordered"
        
        print(f"({phase})")
        
        # Calculate required sweeps
        measure_rate = 500
        nsweeps = measure_rate * n_configs_per_temp
        
        # Generate configurations (NO thermalization!)
        configs = montecarlo(L, T, nsweeps=nsweeps, measure_rate=measure_rate)
        
        # Flatten each configuration and add to dataset
        for config in configs[:n_configs_per_temp]:
            X.append(config.flatten())
            y.append(label)
            temperatures.append(T)
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds() / 60
    print(f"\nCompleted in {duration:.2f} minutes")
    print(f"Total configurations: {len(X)}")
    
    return np.array(X), np.array(y), np.array(temperatures)

# Generate dataset WITHOUT thermalization
print("="*60)
print("ISING MODEL PHASE CLASSIFICATION - NO THERMALIZATION")
print("="*60 + "\n")

X, y, temps = generate_ising_dataset(L=20, n_configs_per_temp=100)

In [ ]:
# Data Analysis and Preparation
print("\n" + "="*60)
print("DATA ANALYSIS")
print("="*60 + "\n")

print(f"Dataset shape: {X.shape}")
print(f"Feature dimension: {X.shape[1]} (from {int(np.sqrt(X.shape[1]))}x{int(np.sqrt(X.shape[1]))} lattice)")
print(f"\nClass distribution:")
print(f"  Ordered (T < Tc):     {np.sum(y == 0)} samples ({100*np.sum(y == 0)/len(y):.1f}%)")
print(f"  Disordered (T > Tc):  {np.sum(y == 1)} samples ({100*np.sum(y == 1)/len(y):.1f}%)")

# Analyze temperature distribution
unique_temps = np.unique(temps)
print(f"\nTemperature range: {unique_temps.min():.3f} to {unique_temps.max():.3f}")
print(f"Number of unique temperatures: {len(unique_temps)}")

# Show sample statistics
print(f"\nSample statistics:")
print(f"  Mean magnetization (ordered):    {np.mean(X[y==0]):.4f}")
print(f"  Mean magnetization (disordered): {np.mean(X[y==1]):.4f}")
print(f"  Std (ordered):    {np.std(X[y==0]):.4f}")
print(f"  Std (disordered): {np.std(X[y==1]):.4f}")

print("\n" + "="*60)
print("NOTICE: Without thermalization, mean magnetizations may be")
print("similar for both classes due to non-equilibrium bias!")
print("="*60)

In [ ]:
# Shuffle and Split Data
print("\n" + "="*60)
print("DATA PREPARATION")
print("="*60 + "\n")

# Shuffle the dataset
np.random.seed(42)
shuffle_idx = np.random.permutation(len(X))
X_shuffled = X[shuffle_idx]
y_shuffled = y[shuffle_idx]
temps_shuffled = temps[shuffle_idx]

# Split into training (80%) and test (20%) sets
X_train, X_test, y_train, y_test, temps_train, temps_test = train_test_split(
    X_shuffled, y_shuffled, temps_shuffled, 
    test_size=0.2, 
    random_state=42,
    stratify=y_shuffled  # Ensure balanced class distribution
)

print(f"Training set: {X_train.shape[0]} samples ({100*len(X_train)/len(X):.0f}%)")
print(f"  Ordered:     {np.sum(y_train == 0)} samples")
print(f"  Disordered:  {np.sum(y_train == 1)} samples")
print(f"\nTest set: {X_test.shape[0]} samples ({100*len(X_test)/len(X):.0f}%)")
print(f"  Ordered:     {np.sum(y_test == 0)} samples")
print(f"  Disordered:  {np.sum(y_test == 1)} samples")

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nStandardization applied:")
print(f"  Training mean: {X_train_scaled.mean():.6f} (should be ~0)")
print(f"  Training std:  {X_train_scaled.std():.6f} (should be ~1)")
print(f"  Test mean:     {X_test_scaled.mean():.6f}")
print(f"  Test std:      {X_test_scaled.std():.6f}")

print("\nData preparation complete!")

In [ ]:
# Visualize Sample Configurations
import matplotlib.pyplot as plt

print("\n" + "="*60)
print("VISUALIZATION")
print("="*60 + "\n")

# Select representative configurations
L = 20
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle('Sample Ising Model Configurations (NO THERMALIZATION)', 
             fontsize=16, fontweight='bold', color='red')

# Show 3 ordered and 3 disordered configurations
ordered_indices = np.where(y_train == 0)[0][:3]
disordered_indices = np.where(y_train == 1)[0][:3]

for i, idx in enumerate(ordered_indices):
    config = X_train[idx].reshape(L, L)
    temp = temps_train[idx]
    axes[0, i].imshow(config, cmap='RdBu_r', vmin=-1, vmax=1)
    axes[0, i].set_title(f'"Ordered" (Non-equilibrium)\nT = {temp:.3f} < Tc', fontsize=10)
    axes[0, i].axis('off')

for i, idx in enumerate(disordered_indices):
    config = X_train[idx].reshape(L, L)
    temp = temps_train[idx]
    axes[1, i].imshow(config, cmap='RdBu_r', vmin=-1, vmax=1)
    axes[1, i].set_title(f'"Disordered" (Non-equilibrium)\nT = {temp:.3f} > Tc', fontsize=10)
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

print("Visualization complete. Blue = spin -1, Red = spin +1")
print("\nNOTE: These configurations may look similar between classes")
print("due to lack of thermalization!")

In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("\n" + "="*60)
print("LOGISTIC REGRESSION CLASSIFIER")
print("="*60 + "\n")

# Train Logistic Regression with default parameters
print("Training Logistic Regression classifier...")
clf = LogisticRegression(random_state=42)
clf.fit(X_train_scaled, y_train)
print("Training complete!\n")

# Make predictions
y_pred_train = clf.predict(X_train_scaled)
y_pred_test = clf.predict(X_test_scaled)

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)

# Calculate metrics for training set (to check for overfitting)
train_accuracy = accuracy_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train)

# Display results
print("TRAINING SET PERFORMANCE:")
print(f"  Accuracy:  {train_accuracy:.4f}")
print(f"  Precision: {train_precision:.4f}")
print(f"  Recall:    {train_recall:.4f}")
print(f"  F1-Score:  {train_f1:.4f}")

print("\nTEST SET PERFORMANCE:")
print(f"  Accuracy:  {test_accuracy:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall:    {test_recall:.4f}")
print(f"  F1-Score:  {test_f1:.4f}")

# Detailed classification report
print("\n" + "-"*60)
print("DETAILED CLASSIFICATION REPORT (Test Set):")
print("-"*60)
print(classification_report(y_test, y_pred_test, 
                          target_names=['Ordered (T<Tc)', 'Disordered (T>Tc)'],
                          digits=4))

# Confusion Matrix
print("-"*60)
print("CONFUSION MATRIX (Test Set):")
print("-"*60)
cm = confusion_matrix(y_test, y_pred_test)
print(f"\n                Predicted")
print(f"              Ordered  Disordered")
print(f"Actual Ordered    {cm[0,0]:3d}      {cm[0,1]:3d}")
print(f"       Disordered {cm[1,0]:3d}      {cm[1,1]:3d}")

# Visualize confusion matrix
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', 
            xticklabels=['Ordered', 'Disordered'],
            yticklabels=['Ordered', 'Disordered'],
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Logistic Regression\n(NO THERMALIZATION)', 
          fontsize=14, fontweight='bold', color='red')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print(f"SUMMARY: Test Accuracy = {test_accuracy:.2%}")
print("="*60)
print("\nEXPECTED: Poor accuracy due to non-equilibrium data!")
print("Without thermalization, the classifier cannot learn meaningful")
print("phase distinctions because the data is contaminated with")
print("non-equilibrium configurations from the random initial state.")

## Analysis: Impact of No Thermalization

### Expected Results

This notebook demonstrates what happens when you **skip thermalization** in Monte Carlo simulations:

#### **1. Non-Equilibrium Bias**

**Problem:**
- Initial state: Random configuration (50% up, 50% down spins)
- Early samples collected before system reaches thermal equilibrium
- Both "ordered" and "disordered" classes contain similar non-equilibrium states

**Result:**
- Mean magnetizations are similar between classes
- Configurations look visually similar regardless of temperature
- No clear phase distinction in the dataset

#### **2. Poor Classification Performance**

**Expected test accuracy: ~50-60%** (barely better than random guessing)

**Why?**
- The classifier correctly identifies that there's **no meaningful difference** between the two classes
- Without equilibration, T=1.5 and T=3.5 configurations are statistically indistinguishable
- Both classes are dominated by the random initial state

#### **3. What This Demonstrates**

This experiment proves that:

✓ **Thermalization is essential** for generating valid statistical mechanics data

✓ **Initial state matters**: Without discarding non-equilibrium samples, the data is biased

✓ **Machine learning detects data quality**: Poor performance indicates contaminated data, not algorithm failure

✓ **Physical validity ≠ numerical stability**: Code runs without errors but produces invalid physics

### Comparison with Proper Thermalization

| Metric | Without Thermalization | With Thermalization |
|--------|------------------------|---------------------|
| Test Accuracy | ~50-60% | ~95%+ (with proper parameters) |
| Ordered Mean Mag | ~0.0 | >>0 (strong ferromagnetic order) |
| Disordered Mean Mag | ~0.0 | ~0 (random) |
| Visual Difference | Minimal | Dramatic |

### Key Takeaway

**The poor performance in this notebook is EXPECTED and CORRECT!**

It demonstrates that:
- Without thermalization, Monte Carlo data is scientifically invalid
- The classifier honestly reports that no phase distinction exists
- This validates the importance of proper equilibration procedures

**Always include thermalization in production Monte Carlo simulations!**